# 🎓 Deploying a ML Model for Online Inference with Vertex AI - (Part 2)

### Introduction

In this notebook, we will walk through the complete end-to-end process of deploying a machine learning model on **Google Cloud** for online inference.

Our goal is to take a pre-trained logistic regression model, designed to predict the sentiment (positive or negative) of customer reviews for women's clothing, and make it available as a live, scalable service using Vertex AI.

### Workflow

This notebook is part 2 of this process, and it basically contains the following steps:

3.  **Importing to Vertex AI Model Registry:** We will upload our `model.joblib` artifact to Google Cloud and register it in the Vertex AI Model Registry, which acts as a central repository for our models.

4.  **Deploying to an Endpoint:** Once the model is registered, we will deploy it to a Vertex AI Endpoint. This creates a live, public URL that applications can use to access our model.

5.  **Getting Online Inferences:** This notebook will demonstrate how to use the Vertex AI Python SDK to send a new review to our deployed endpoint and receive a real-time sentiment prediction (i.e. online inferences).

In [1]:
from google.cloud import aiplatform
import joblib

In [ ]:
# 1. Initialize with your Project ID (this is the best practice)
aiplatform.init(project='my-project-22345', location='asia-southeast1')

# 2. Create the Endpoint object.
# You can provide just the ID number as a string. The library will
# use the project and location from init() to build the full path.
endpoint = aiplatform.Endpoint(
    endpoint_name="7810580958323237232"  # endpoint_id
)

# Now you can use the endpoint object, for example:
# response = endpoint.predict(...)
print("Successfully created Endpoint object.")
print(f"Endpoint resource name: {endpoint.resource_name}")

Successfully created Endpoint object.
Endpoint resource name: projects/790592728786/locations/asia-southeast1/endpoints/7810580959023071232


In [3]:
# This is the new review we want to classify
new_review = ["The material felt cheap and it was not what I expected."]
print(f"New review to classify: '{new_review[0]}'")

New review to classify: 'The material felt cheap and it was not what I expected.'


In [4]:
# --- Step 6: Preprocess the New Review for Prediction ---

# Load vectorizer
vectorizer = joblib.load("vectorizer.joblib")

# Step 1: Transform text into a sparse matrix
# The vectorizer uses its learned vocabulary to count word occurrences.
# The output is a highly efficient "sparse matrix" format.
print("Step 1: Creating a sparse matrix from the text...")
sparse_matrix = vectorizer.transform(new_review)
print(f"Sparse matrix shape: {sparse_matrix.shape}")

Step 1: Creating a sparse matrix from the text...
Sparse matrix shape: (1, 11897)


In [5]:
# Step 2: Convert the sparse matrix to a dense NumPy array
# This turns the efficient sparse format into a standard numerical array
# that's easier to work with.
print("Step 2: Converting to a dense NumPy array...")
numpy_array = sparse_matrix.toarray()

# Step 3: Convert the NumPy array to a standard Python list
# The Vertex AI endpoint requires the data in a JSON-friendly format,
# which is a standard Python list.
print("Step 3: Converting to a final Python list for the API...")
processed_review = numpy_array.tolist()

# The final 'processed_reviews' variable is now ready to be sent to the model.
print("\nFinal list ready for prediction:", processed_review)

Step 2: Converting to a dense NumPy array...
Step 3: Converting to a final Python list for the API...

Final list ready for prediction: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
print("\nSending prediction request to the Vertex AI endpoint...")
# Make the prediction call
response = endpoint.predict(instances=processed_review)

print(response)
print(type(response))


Sending prediction request to the Vertex AI endpoint...
Prediction(predictions=[-1], deployed_model_id='5205864301100793856', metadata=None, model_version_id='1', model_resource_name='projects/790592728786/locations/asia-southeast1/models/6119534274526838784', explanations=None)
<class 'google.cloud.aiplatform.models.Prediction'>


In [7]:
print(response.predictions)
print(type(response.predictions))

[-1]
<class 'list'>


In [8]:
new_reviews = [
    "I am so disappointed with this purchase, I will be returning it.",
    "The material felt cheap and it was not what I expected.",
    "It's an okay product, not great but not terrible either.",
    "This dress is absolutely beautiful and fits perfectly!",
    "I recently purchased this dress and I have to say, it exceeded my expectations. The fabric feels soft yet durable, and it has just the right amount of stretch to make it really comfortable for long wear. The stitching and finishing are neat, giving it a very polished look."
]

print("Transforming the new review into a numerical vector...")
processed_reviews = vectorizer.transform(new_reviews).toarray().tolist()

print("\nSending prediction request to the Vertex AI endpoint...")
# Make the prediction call
response = endpoint.predict(instances=processed_reviews)

print(response)
print(type(response))

prediction_results = response.predictions

print(prediction_results)

Transforming the new review into a numerical vector...

Sending prediction request to the Vertex AI endpoint...
Prediction(predictions=[-1, -1, -1, 1, 1], deployed_model_id='5205864301100793856', metadata=None, model_version_id='1', model_resource_name='projects/790592728786/locations/asia-southeast1/models/6119534274526838784', explanations=None)
<class 'google.cloud.aiplatform.models.Prediction'>
[-1, -1, -1, 1, 1]
